# Preprocessing

The raw database, as provided by the medical instituion was difficult to process. Files were in different formats, variables refering to the same thing were named differently across tables, data types were not consistent, etc.

In this notebook the raw tables have been transformed in a consistent way to make more efficient the later handling of the data. The final output is a directory containing all the hospital tables in .parquet files with columns named consistently across tables and appropiate data types for each variable of the dataset.

### Import requirements

In [ ]:
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', None)  
from thesis_lib.data_processing import *
from datetime import datetime

### Upload data

In [2]:
process_raw_files('SanatorioGüemes','data')

labos.csv


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2856: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


labos.csv successfully copied to data/labos.csv
internaciones.xlsx
internaciones.xlsx successfully converted to csv: data/internaciones.csv
ingresos_sectores.xlsx
ingresos_sectores.xlsx successfully converted to csv: data/ingresos_sectores.csv
cirugias.xlsx
cirugias.xlsx successfully converted to csv: data/cirugias.csv
sectores.xlsx
sectores.xlsx successfully converted to csv: data/sectores.csv
images.xlsx
images.xlsx successfully converted to csv: data/images.csv


In [2]:
db = get_database('data')

Loading dataset:  images
Loading dataset:  laboratories


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loading dataset:  hospital_sectors
Loading dataset:  sectors_admissions
Loading dataset:  surgeries


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (74,77) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loading dataset:  hospitalizations


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (24,43,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Hospitalizations dataset

##### Rename columns

In [ ]:
df = db['hospitalizations']

In [7]:
series_names = {
'Nro Adm': 'admission_id', 
'Nro H.C.': 'patient_id', 
'Edad': 'age', 
'Sexo': 'gender', 
'Fec Nac': 'birth_date', 
'Entidad': 'insurance_entity',
'AgrupEntidad': 'entity_group' , 
'Fec Adm': 'admission_date', 
'Hora Adm': 'admission_time', 
'AñoAdm': 'admission_year', 
'MesAdm': 'admission_month',
'Procedencia': 'origin', 
'Médico Admisión': 'admission_physician', 
'SecAdmisión': 'admission_sector', 
'SecUltimo': 'last_sector',
'CategUlt': 'last_category', 
'Aislación': 'isolation', 
'HabitUlt': 'last_room', 
'CamaUlt': 'last_bed', 
'Fec Alta': 'discharge_date', 
'Hora Alta': 'discharge_time',
'AñoAlta': 'discharge_year', 
'MesAlta': 'discharge_month', 
'Motivo Alta': 'discharge_reason', 
'PreAlta': 'pre_discharge_date', 
'1erSecInt': 'first_sector',
'Diagnóstico Administrativo No Codificado': 'administrative_diagnosis', 
'CodDiagPresu': 'diagnosis_code',
'Diagnóstico Presuntivo': 'presumptive_dianogsis',
'CodCieDiagEgr': 'discharge_diagnosis_code', 
'Diagnóstico Egreso': 'discharge_diagnosis',
'FecDenunEgre': 'date_registered_discharge', 
'HoraDenunEgre': 'time_registered_discharge', 
'Médico Denuncia Egreso': 'discharge_physician', 
'Epicrisis': 'discharge_summary',
'MédicoEpicrisis': 'discharge_summary_physician', 
'Quirurg': 'surgery', 
'CaderaExpress': 'express_hip_surgery',
'Permanencia': 'admission_lenght_days',
'Servicio Responsable': 'responsible_sector',
'Servicio Co-Responsable': 'second_responsible_sector', 
'FecHorIngGua': 'emergency_admission_datetime',
'Prest.Guardia': 'emergency_service', 
'Reingreso': 'has_previous_admission', 
'AdmAntReciente':  'previous_admission_id' ,
'FecAdmAntReciente': 'previous_admission_date', 
'FecAltaAntReciente': 'previous_discharge_date', 
'SecAntReciente': 'previous_sector',
'Diagn. Egreso Admisión Anterior Reciente': 'previous_discharge_dianosis', 
'AmbulanciaEgreso': 'discharge_ambulance',
'PesoAlNacer': 'new_born_weight', 
'EdadGestac': 'new_born_gestation_age',
'PIM2TEP': 'PIM2TEP', 
'DiagAltoRiesgoTEP': 'high_risk_TEP',
'DiagBajoRiesgoTEP': 'low_risk_TEP', 
'ARM_TEP': 'ARM_TEP', 
'CEC_TEP': 'CEC_TEP', 
'SolicDerivación': 'request_number',
'OrigDerivación': 'request_origin',
'Procedencia.1': 'request', 
'DiagnósticoDerivación': 'request_diagnosis',
'AreaDerivación': 'request_sector',
'Notificado': 'notified', 
'UsuarioDeriv': 'request_user'}

In [8]:
df.rename(series_names,axis=1, inplace=True)

##### Convert columns

Boolean columns

In [10]:
boolean_cols = ['discharge_summary',
'surgery',
'express_hip_surgery',
'has_previous_admission',
'discharge_ambulance',
'ARM_TEP',
'CEC_TEP']

In [11]:
for col in boolean_cols:
    df[col] = df[col].apply(lambda x : True if x == 'Sí' else  (False if x == 'No' else None ))

##### Drop duplicated columns

In [12]:
columns_to_drop = ['Prestac.Guardia']

In [13]:
df.drop(columns_to_drop,axis=1, inplace=True)

In [15]:
len(df.patient_id.unique())

64507

In [17]:
len(df.loc[~df['discharge_date'].isna()].patient_id.unique())

64156

##### Delete invalid rows

In [18]:
df = df.loc[~df['discharge_date'].isna()]

##### Generate datetime cols

In [21]:
df['admission_datetime'] = pd.to_datetime(df['admission_date'].map(str)+" "+
                                            df['admission_time'].map(str)
                                            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
df['discharge_datetime'] = pd.to_datetime(
                                           df['discharge_date'].map(str)+" "+
                                            df['discharge_time'].map(str),
                                            errors='ignore'
                                            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


##### Save as .parquet

In [27]:
df.to_parquet('hospitalizations.parquet')

### Laboratories dataset

##### Rename columns

In [17]:
series_names = { 
    'Nro Vale': 'labo_id',
    'Pun': 'labo_pun',
    'Estado': 'status',
    'Fecha': 'labo_date', 
    'Hora': 'labo_time',
    'AñoMes':'labo_year-month',
    'Sector': 'sector',
    'Nro Adm': 'admission_id',
    'HistClín': 'patient_id',
    'Fec.Adm.': 'admission_date',
    'Fec.Alta': 'discharge_date',
    'Entidad': 'entity_id',
    'Nombre Entidad': 'insurance_entity',
    'Nro.Afiliado': 'entity_affiliate_id',
    'GrupoEnt': 'entity_group',
    'Urgencia': 'emergency',
    'Nombre del Solicitante': 'requester_name',
    'Función del Solicitante': 'requester_role',
    'Prestación': 'study_code',
    'Descrip Prestación': 'study_description',
    'CantPrest': 'no_of_studies',
    }

In [18]:
db['laboratories'].rename(series_names,axis=1, inplace=True)

In [3]:
db['laboratories']['Servicio'].unique()

array(['LABORATORIO'], dtype=object)

##### Drop cols

In [20]:
columns_to_drop = ['Nombre Paciente',
                   'HH',
                   'Mnemo Serv',
                   'Cod Serv',
                   'Cod Insumo',
                   'Descrip Insumo',
                   'CantInsumos',
                   'EstadoResultado',
                   'Observ.Estudio']

In [21]:
db['laboratories'].drop(columns_to_drop,axis=1, inplace=True)

##### Save as .parquet

In [23]:
db['laboratories'].to_parquet('laboratory.parquet')

### Images dataset

##### Rename columns

In [15]:
series_names = { 
    'Nro Vale': 'image_id',
    'Pun': 'image_pun',
    'Estado': 'status',
    'Fecha': 'image_date', 
    'Hora': 'image_time',
    'AñoMes':'image_year-month',
    'Sector': 'sector',
    'Nro Adm': 'admission_id',
    'HistClín': 'patient_id',
    'Fec.Adm.': 'admission_date',
    'Fec.Alta': 'discharge_date',
    'Entidad': 'entity_id',
    'Nombre Entidad': 'insurance_entity',
    'Nro.Afiliado': 'entity_affiliate_id',
    'GrupoEnt': 'entity_group',
    'Urgencia': 'emergency',
    'Nombre del Solicitante': 'requester_name',
    'Función del Solicitante': 'requester_role',
    'Prestación': 'study_code',
    'Descrip Prestación': 'study_description',
    'CantPrest': 'no_of_studies',
     'Servicio': 'type_of_service'}

In [16]:
db['images'].rename(series_names,axis=1, inplace=True)

##### Drop columns

In [17]:
columns_to_drop = [
                   'HH',
                   'Mnemo Serv',
                   'Cod Serv',
                   'Cod Insumo',
                   'Descrip Insumo',
                   'CantInsumos',
                   'Observ.Estudio']

In [18]:
db['images'].drop(columns_to_drop,axis=1, inplace=True)

##### Save as .parquet

In [20]:
db['images'].to_parquet('images.parquet')

### Hospital Sectors dataset

##### Rename columns

In [33]:
series_names = { 
   'CodSector': 'sector_code',
    'NombreSector': 'sector_name',
    'TipoSector': 'sector_type'}

In [34]:
db['hospital_sectors'].rename(series_names,axis=1, inplace=True)

##### Save as .parquet

In [36]:
db['hospital_sectors'].to_parquet('hospital_sectors.parquet')

### Sectors admissions dataset

##### Rename columns

In [62]:
series_names = { 
   'Nro Adm': 'admission_id', 
    'Nro H.C.': 'patient_id', 
    'FecIngrSec': 'sector_admission_date', 
    'HoraIngrSec': 'sector_admission_time', 
    'Sector': 'sector_code', 
    'Categ': 'category'}

In [63]:
db['sectors_admissions'].rename(series_names,axis=1, inplace=True)

##### Generate datetime columns

In [69]:
db['sectors_admissions']['sector_admission_datetime']  = pd.to_datetime(db['sectors_admissions'].sector_admission_date.map(str)+" "+db['sectors_admissions'].sector_admission_time.map(str))

In [70]:
db['sectors_admissions'].head()

,admission_id,patient_id,sector_admission_date,sector_admission_time,sector_code,category,sector_admission_datetime
0,466470-1,3526141-4,2017-01-01,02:00:00,EME,I,2017-01-01 02:00:00
1,466470-1,3526141-4,2017-01-01,07:53:00,T16,C,2017-01-01 07:53:00
2,466471-4,3714597-2,2017-01-01,02:00:00,EME,I,2017-01-01 02:00:00
3,466471-4,3714597-2,2017-01-01,06:38:00,UCO,I,2017-01-01 06:38:00
4,466472-7,3714595-2,2017-01-01,02:00:00,T10,C,2017-01-01 02:00:00


##### Save as .parquet

In [71]:
db['sectors_admissions'].to_parquet('sectors_admissions.parquet')

### Surgeries

##### Rename columns

In [26]:
series_names = {'Quirófano' : 'operating_room', 
                'FechaQuirof' : 'surgery_date', 
                'DíaSem': 'surgery_weekday', 
                'AñoMes': 'surgery_year_month',
                'HoraProgram': 'surgery_scheduled_time', 
                'CodAdmision': 'admission_id', 
                'UrgProgProtoc': 'surgery_type',
                'IdCirugia': 'surgery_id',
                'DescProgOri': 'origin' ,
                'HistClínica': 'patient_id',
                'Sexo': 'gender', 
                'Edad': 'age', 
                'Motivo Alta': 'discharge_type',
                'DescripEntidad': 'entity_description', 
                'Diagnostico': 'diagnosis',
                'Operación Programada': 'scheduled_surgery',
                'Operación en Parte Quirúrgico': 'actual_surgery', 
                'Cirujano': 'surgery_physician', 
                'Dependencia': 'dependency',
                'DescripTipoAnestesia': 'anesthesia_type',  
                'Antisepsia': 'antisepsia', 
                'ATB profiláctico':'prophylactic_ATB',
                'Dosis mg': 'dosis_mg', 
                'CamaSolic': 'bed_request', 
                'Hemoterapia': 'hemotherapy' , 
                'HemoOk': 'hemo_ok', 
                'Rayos': 'x_ray' , 
                'Cardiologo':'cardiologist' ,
                'Material': 'supplies',
                'MateOk': 'supplies_ok', 
                'NroProtocolo': 'protocol_no', 
                'DescripServicio': 'service_description', 
                'OperCod': 'surgery_code',
                'CamaSector': 'sector_bed', 
                'DescripEspecialidad':'specialization',
                'DuracionEstimada':'estimated_duration', 
                'HoraInic': 'surgery_startime',
                'HoraFin': 'surgery_endtime', 
                'DemoraInicio': 'surgery_delay',
                'FechaInternac': 'hospitalization_date', 
                'EspecPrestador': 'specialization_code',
                'HoraIngre': 'entry_time' , 
                'HoraEgre': 'exit_time', 
                'HoraIndAnes': 'anesthesia_startime', 
                'HoraFinAnes': 'anesthesia_endtime', 
                'PostOper':  'post_surgery_condition',
                'FechaAdmision' : 'admission_date', 
                'HoraAdm': 'admission_time', 
                'FechaAlta': 'discharge_date',
                'HoraAlta' : 'discharge_time', 
                'ReCirug': 're_surgery', 
                'CantCirug': 'no_of_surgeries', 
                'CaderaExpress': 'hips_surgery',
                'TipoHerida': 'injury_condition' ,
                'DuracionPreCirugia': 'pre_surgery_duration',
                'DuracionCirugia': 'surgery_duration',
                'DuracionPostCirugia': 'post_surgery_duration',
                'DuracionPreparacion': 'surgery_prep_duration', 
                'Reinternación': 're_admission_id',
                'Antibiot':'antibiotic' ,
                'Cultivo': 'seeding', 
                'CntAyud': 'no._of_assistans', 
                'MatricAnest': 'anesthetist_id', 
                'Gestac': 'no._of_pregnancies' , 
                'Partos': 'no._births',
                'Cesár.': 'no._of_cesarean', 
                'Nulípara':'nulliparous', 
                'IdRelRN': 'new_born_id_rel', 
                'NroAdmRN': 'new_born_admission_id', 
                'PesoRecNac': 'new_born_weight', 
                'EdadGestac': 'new_born_gestation_age', 
                'BebeEgresoVivo': 'new_born_alive', 
                'BactPosit': 'bact_positive'}

In [27]:
db['surgeries'].rename(series_names,axis=1, inplace=True)

##### Drop columns

In [ ]:
columns_to_drop = ['EstadoAgrup','DíasIntern','HhIndAnes','HhInicio','TorreLapar']

In [29]:
db['surgeries'].drop(columns_to_drop,axis=1, inplace=True)

##### Format boolean columns

In [30]:
boolean_cols = ['G.A.P.','bed_request', 'hemotherapy', 'hemo_ok', 'x_ray', 'cardiologist', 'supplies', 'supplies_ok','hips_surgery', 
'nulliparous']

In [31]:
for col in boolean_cols:
    db['surgeries'][col] = db['surgeries'][col].apply(lambda x : True if x == 'Sí' else  (False if x == 'No' else None ))

###### Generate new columns

In [41]:
duration_cols = ['estimated_duration',
'pre_surgery_duration',
'surgery_duration',
'post_surgery_duration',
'surgery_prep_duration']

In [45]:
for col in duration_cols:
    
    minutes_duration_series = []
    for i in pd.to_datetime(db['surgeries'][col], errors='ignore').dt.time:
        minutes_duration_series.append(i.hour*60+i.minute)
    
    db['surgeries'][col] = pd.Series(minutes_duration_series)

In [48]:
db['surgeries']['scheduled_surgery_done'] = db['surgeries'].scheduled_surgery == db['surgeries'].actual_surgery

##### Save as .parquet

In [56]:
db['surgeries'].to_parquet('surgeries.parquet')